In [ ]:
import numpy as np
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torchvision import datasets
from torchvision.transforms import ToTensor

In [ ]:
from model import Net
from supconloss import SupCon

In [ ]:
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
BATCH_SIZE = 64
EPOCHS = 10

In [ ]:
# TODO: add transformations/augmentations/normalisation
training_data = datasets.CIFAR10(root="data", train=True, download=True, transform=ToTensor())
train_loader = torch.utils.data.DataLoader(training_data, batch_size=BATCH_SIZE, shuffle=True, num_workers=8)
# TODO: divide into train and valid

test_data = datasets.CIFAR10(root="data", train=False, download=True, transform=ToTensor())
test_loader = torch.utils.data.DataLoader(test_data, batch_size=BATCH_SIZE, shuffle=True, num_workers=8)

classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

In [ ]:
# it = iter(train_loader) # create an iterator over the DataLoader
# first = next(it)
# print(first[1]) # in position [1], we get the vector of the labels of the batch
# second = next(it)
# print(second[0].shape)

In [ ]:
labels_map = {
    0: "Airplane",
    1: "Automobile",
    2: "Bird",
    3: "Cat",
    4: "Deer",
    5: "Dog",
    6: "Frog",
    7: "Horse",
    8: "Ship",
    9: "Truck",
}
figure = plt.figure(figsize=(8, 8))
cols, rows = 3, 3
for i in range(1, cols * rows + 1):
    sample_idx = torch.randint(len(training_data), size=(1,)).item()
    img, label = training_data[sample_idx]
    figure.add_subplot(rows, cols, i)
    plt.title(labels_map[label])
    plt.axis("off")
    plt.imshow(img.permute(1, 2, 0))
plt.show()

In [ ]:
training_data.data.shape

In [ ]:
train_loader.dataset.data.shape

`training_data` is the Dataset, whereas `train_loader` is the wrapper around the Dataset and it controls how data is delivered by creating mini-batches, shuffling, workers, iterating, ... $\rightarrow$ `train_loader.dataset` is the Dataset  
The dataset is composed of pairs of (image, class), where the image is a matrix of pixels and the class $\in \{0,9\}$

In [ ]:
train_loader.dataset[15][1] # class

In [ ]:
classic_model = Net().to(DEVICE)
combined_model = Net().to(DEVICE)

In [ ]:
# optimizer = optim.Adam(model.parameters(), lr=0.001)
optimizer1 = optim.SGD(classic_model.parameters(), lr=0.01, momentum = 0.9)
optimizer2 = optim.SGD(combined_model.parameters(), lr=0.01, momentum = 0.9)
criterion1 = nn.CrossEntropyLoss()
criterion2 = SupCon()
total_step = len(train_loader)

Allora, l'idea e' quella di implementare la contrastive loss. Cio' significa ridurre la distanza tra vettori simili e aumentare la distanza tra vettori diversi $\rightarrow$ loss in funzione della distanza tra i due vettori: se appartenenti alla stessa classe e bassa distanza, loss bassa, e viceversa
Contrastive Loss: $$ L_{i, j} = -\log \frac{\exp(sim(z_i, z_j) / \tau)}{\sum_{k\neq i}\exp(sim(z_i, z_k) / \tau)}, \quad\quad\quad \text{where } sim(z_i, z_j) = \frac{z_i\cdot z_j}{||z_i|| \ ||z_j||} $$

SCHERZONE QUESTA È la NT-XENT nooooooo  
Noi vogliamo la SupCon: $$ \mathcal L_i = {-1\over |P(i)|}\sum_{p\in P(i)}\log \frac{\exp(sim(z_i, z_p) / \tau)}{\sum_{a\neq i}\exp(sim(z_i, z_a) / \tau)}, $$ where $P(i)$ is the set of positives for anchor $i$.
$\rightarrow$ final loss: $$ L = {1\over B}\sum_i L_i $$

In [ ]:
for epoch in range(EPOCHS):
    for i, (images, labels) in enumerate(train_loader): # iterating over all the batches
        # Move tensors to the configured device
        images = images.to(DEVICE)
        labels = labels.to(DEVICE)
        optimizer1.zero_grad()
        
        # Forward pass
        outputs = classic_model(images)
        loss = criterion1(outputs, labels)
        
        # Backward and optimize
        loss.backward()
        optimizer1.step()

    print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}' 
                   .format(epoch+1, EPOCHS, i+1, total_step, loss.item()))

In [ ]:
for epoch in range(EPOCHS):
    for i, (images, labels) in enumerate(train_loader): # iterating over all the batches
        # Move tensors to the configured device
        images = images.to(DEVICE)
        labels = labels.to(DEVICE)
        optimizer2.zero_grad()
        
        # Forward pass
        outputs = combined_model(images)
        loss = criterion2(outputs, labels)
        
        # Backward and optimize
        loss.backward()
        optimizer2.step()

    print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}' 
                   .format(epoch+1, EPOCHS, i+1, total_step, loss.item()))

In [ ]:
classic_model.eval()
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images = images.to(DEVICE)
        labels = labels.to(DEVICE)
        outputs = classic_model(images)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        del images, labels, outputs

    print('Accuracy of the network on the {} test images: {} %'.format(10000, 100 * correct / total))